# Filtado de mensajes spam

## Descripción del problema real

La recepción de publicidad no deseada a traves mensajes de texto usando SMS (Short Message Service) es un problema que afecta a muchos usuarios de teléfonos móviles. El problema radica en que los usuarios deben pagar por los mesajes recibidos, y por este motivo resulta muy importante que las compañías prestadoras del servicio puedan filtrar mensajes indeseados antes de enviarlos a su destinatario final. Los mensajes tienen una longitud máxima de 160 caracteres, por lo que el texto resulta poco para realizar la clasificación, en comparación con textos más largos (como los emails). Adicionalmente, los errores de digitación dificultan el proceso de detección automática.

## Descripción del problema en términos de los datos

Se tiene una muestra contiene 5574 mensajes en inglés, no codificados y clasificados como legítimos (ham) o spam (http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/). La información está almacenada en el archivo `datos/spam-sms.zip`.El problema en términos de los datos consiste en clasificar si un mensaje SMS es legítico o spam, a partir del análisis de las palabras que contiente, partiendo del supuesto de que ciertas palabras que son más frecuentes dependiendo del tipo de mensaje. Esto implica que en la fase de preparación de los datos se deben extraer las palabras que contiene cada mensaje para poder realizar el análsis.

## Aproximaciones posibles

En este caso, se desea comparar los resultados de un modelo de redes neuronales artificiales y otras técnicas estadísticas para realizar la clasificación.

## Requerimientos

Usted debe:

* Preprocesar los datos para representarlos usando bag-of-words.


* Construir un modelo de regresión logística como punto base para la comparación con otros modelos más complejos.


* Construir un modelo de redes neuronales artificiales. Asimismo, debe determinar el número de neuronas en la capa o capas ocultas.


* Utiizar una técnica como crossvalidation u otra similar para establecer la robustez del modelo.


* Presentar métricas de desempeño para establecer las bondades y falencias de cada clasificador.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
import nltk
import string
import glob
import re

## Lectura de datos

In [ ]:
df = pd.DataFrame(columns = ['label', 'content'])

folders = glob.glob("./datos/spam-filter/*")
for folder_aux in folders:
    folder_name = folder_aux + "/*"
    folder = glob.glob(folder_name)
    for file in folder:
        try:
            with open(file, encoding='utf-8') as f:
                content = f.read()
        except UnicodeDecodeError:
            with open(file, 'rb') as f:
                content = f.read()
        df = df.append({'label': 0 if 'ham' in folder_name else 1, 
                        'content': str(content).replace('\\n', '\n')}, ignore_index=True)
        
df.to_csv(r'datos/spam-filter/dataFrame.csv')

def remove_header(cols):
    array_first = cols[1].replace('\\n', '\n').split('\n')
    index_remove = 0
    pattern = r'^[A-Za-z-]*: [a-zA-Z 0-9@.[\]\(\)-]*'
    for index, element in enumerate(array_first):
        if re.search(pattern, element):
            index_remove = index
    return '\n'.join(array_first[index_remove + 1:]).strip()

def remove_tags_HTML(cols):
    content = cols[1]
    TAG_RE = re.compile(r'<[^>]+>')
    TAG_REX = re.compile(r"\s+")
    content = TAG_RE.sub('', content)
    content = TAG_REX.sub(' ', content)
    return content

def remove_spaces_tab(cols):
    content = cols[1]
    return content.replace("\n","").strip("\t")

def remove_web(cols):
    content = cols[1]
    TAG_RE = re.compile(r'https?:\/\/.*[\n]*')
    return TAG_RE.sub('', content)

def remove_numbers(cols):
    content = cols[1]
    TAG_RE = re.compile(r'[0-9]*')
    return TAG_RE.sub('', content)

df['content'] = df.apply(remove_header, axis=1)
df['content'] = df.apply(remove_tags_HTML, axis=1)
df['content'] = df.apply(remove_spaces_tab, axis=1)
df['content'] = df.apply(remove_numbers, axis=1)
df['content'] = df.apply(remove_web, axis=1)

X = df['content']
y = df['label']

## Limpieza del texto

In [ ]:
stemmer = SnowballStemmer("english")
nltk.download('stopwords')

def cleanText(message):    
    message = message.translate(str.maketrans('', '', string.punctuation))
    words = [stemmer.stem(word) for word in message.split() if word.lower() not in stopwords.words("english")]
    return " ".join(words)
    
X = list(map(cleanText, X))

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer 

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(X)

X.shape

In [ ]:
X = X.toarray()

X[0]